In [1]:
%matplotlib inline

import os
import shutil
import time

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import DataLoader
from densenet_modified import densenet201

In [2]:
# Trainer parameters
print_freq_epochs = 100
use_cuda = True

# Dataset Parameters
batch_size = 64
load_size = 128
fine_size = 128
c = 3
data_mean = np.asarray([0.45834960097,0.44674252445,0.41352266842])

# Training parameters
# architecture = 'resnet34'
architecture = 'dense'
lr = 0.1  # Can be large for resnet or if using batch norm
momentum = 0.90
weight_decay = 1e-4
num_epochs = 100

In [3]:
def construct_dataloader_disk():
    # Construct DataLoader
    opt_data_train = {
        #'data_h5': 'miniplaces_128_train.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        #'data_h5': 'miniplaces_128_val.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoader.DataLoaderDisk(**opt_data_train)
    loader_val = DataLoader.DataLoaderDisk(**opt_data_val)
    
    return (loader_train, loader_val)

def construct_dataloader_h5():
    # Construct DataLoader from an h5 file
    opt_data_train = {
        'data_h5': 'miniplaces_128_train.h5',
        #'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        #'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        'data_h5': 'miniplaces_128_val.h5',
        #'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        #'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoader.DataLoaderH5(**opt_data_train)
    loader_val = DataLoader.DataLoaderH5(**opt_data_val)
    
    return (loader_train, loader_val)

In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def adjust_learning_rate(lr, optimizer, epoch):
    """Calculates a learning rate of the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def save_checkpoint(model, state, is_best, epoch):
    torch.save(state, "models/"+"checkpoint_dense201.tar")
    if is_best:
        torch.save(model, "models/"+"best_dense201.pt")

In [5]:
# train and validate methods adapted from https://github.com/pytorch/examples/blob/master/imagenet/main.py

def train(train_loader, model, criterion, optimizer, epoch, text_file):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i in range(int(train_loader.size()/batch_size)):
        input, target = train_loader.next_batch(batch_size)
        target = target.long()
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
        target_var = target_var.long()
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        text_file.write(str(epoch)+str(",")+str(i)+str(",")+str(batch_time.val)+str(",")+str(data_time.val)+str(",")+str(losses.val)+str(",")+str(top1.val)+str(",")+str(top5.val)+"\n")
        
        if i % print_freq_epochs == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, train_loader.size()/batch_size, batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))


def validate(val_loader, model, criterion, text_file):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(int(val_loader.size()/batch_size)):
        input, target = val_loader.next_batch(batch_size)
        target = target.long()
        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        target_var = target_var.long()

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        text_file.write(str("val, ")+str(i)+str(",")+str(batch_time.val)+str(",")+str(losses.val)+str(",")+str(top1.val)+str(",")+str(top5.val)+"\n")
        
        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))


    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top5.avg

In [10]:
if architecture[:5]!='dense':
    model = models.__dict__[architecture]()
else:
    model = densenet201()
    
criterion = nn.CrossEntropyLoss()

if use_cuda:
    criterion = criterion.cuda()
    model = model.cuda()

In [11]:
# model.avgpool = nn.AdaptiveAvgPool2d(1) # to be used for ResNets 

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [13]:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=weight_decay)

In [14]:
# train_loader, val_loader = construct_dataloader_disk()
train_loader, val_loader = construct_dataloader_h5()

# Images found: 100000
# Images found: 10000


In [9]:
# model = torch.load('resnet504-3.0.ph')

In [15]:
best_prec5 = 0.

In [12]:
for epoch in range(1,num_epochs):
    lr = adjust_learning_rate(lr, optimizer, epoch)

    text_file = open("epoch_output_densenet201.txt", "w")
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, text_file)

    # evaluate on validation set
    prec5 = validate(val_loader, model, criterion, text_file)
    text_file.close()
    
    # remember best prec@1 and save checkpoint
    is_best = prec5 > best_prec5
    best_prec5 = max(prec5, best_prec5)
    save_checkpoint(model, {
        'epoch': epoch + 1,
        'arch': architecture,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec5,
        'optimizer' : optimizer.state_dict(),
    }, is_best, epoch)

Epoch: [1][0/1562.5]	Time 3.479 (3.479)	Data 3.231 (3.231)	Loss 2.7848 (2.7848)	Prec@1 28.125 (28.125)	Prec@5 54.688 (54.688)
Epoch: [1][100/1562.5]	Time 0.262 (0.296)	Data 0.025 (0.058)	Loss 2.8803 (3.1025)	Prec@1 26.562 (22.200)	Prec@5 62.500 (51.485)
Epoch: [1][200/1562.5]	Time 0.262 (0.280)	Data 0.025 (0.042)	Loss 3.0842 (3.0971)	Prec@1 25.000 (22.683)	Prec@5 54.688 (51.920)
Epoch: [1][300/1562.5]	Time 0.261 (0.274)	Data 0.025 (0.037)	Loss 3.2505 (3.0740)	Prec@1 18.750 (23.007)	Prec@5 57.812 (52.435)
Epoch: [1][400/1562.5]	Time 0.262 (0.271)	Data 0.025 (0.034)	Loss 3.1500 (3.0658)	Prec@1 17.188 (23.130)	Prec@5 51.562 (52.782)
Epoch: [1][500/1562.5]	Time 0.262 (0.269)	Data 0.025 (0.032)	Loss 3.1666 (3.0595)	Prec@1 18.750 (23.282)	Prec@5 54.688 (53.078)
Epoch: [1][600/1562.5]	Time 0.262 (0.268)	Data 0.025 (0.031)	Loss 2.9420 (3.0432)	Prec@1 23.438 (23.560)	Prec@5 56.250 (53.494)
Epoch: [1][700/1562.5]	Time 0.268 (0.268)	Data 0.029 (0.030)	Loss 2.9796 (3.0278)	Prec@1 21.875 (23.921)	P

Epoch: [4][1100/1562.5]	Time 0.263 (0.267)	Data 0.026 (0.029)	Loss 2.4685 (2.3094)	Prec@1 34.375 (39.149)	Prec@5 68.750 (70.819)
Epoch: [4][1200/1562.5]	Time 0.262 (0.266)	Data 0.025 (0.029)	Loss 1.8947 (2.3093)	Prec@1 53.125 (39.122)	Prec@5 81.250 (70.832)
Epoch: [4][1300/1562.5]	Time 0.262 (0.266)	Data 0.025 (0.029)	Loss 2.1563 (2.3137)	Prec@1 48.438 (39.005)	Prec@5 73.438 (70.790)
Epoch: [4][1400/1562.5]	Time 0.263 (0.266)	Data 0.026 (0.029)	Loss 2.6108 (2.3142)	Prec@1 34.375 (39.026)	Prec@5 57.812 (70.769)
Epoch: [4][1500/1562.5]	Time 0.262 (0.266)	Data 0.026 (0.028)	Loss 2.1039 (2.3130)	Prec@1 46.875 (39.006)	Prec@5 75.000 (70.791)
Test: [0/156.25]	Time 0.103 (0.103)	Loss 2.9393 (2.9393)	Prec@1 29.688 (29.688)	Prec@5 54.688 (54.688)
Test: [100/156.25]	Time 0.102 (0.102)	Loss 2.6251 (2.6185)	Prec@1 28.125 (33.431)	Prec@5 65.625 (63.691)
 * Prec@1 33.914 Prec@5 64.253
Epoch: [5][0/1562.5]	Time 0.265 (0.265)	Data 0.029 (0.029)	Loss 2.0742 (2.0742)	Prec@1 43.750 (43.750)	Prec@5 75.000

Epoch: [8][400/1562.5]	Time 0.268 (0.276)	Data 0.027 (0.035)	Loss 1.8442 (1.9026)	Prec@1 53.125 (48.122)	Prec@5 75.000 (79.095)
Epoch: [8][500/1562.5]	Time 0.267 (0.275)	Data 0.026 (0.034)	Loss 2.2695 (1.9094)	Prec@1 37.500 (47.910)	Prec@5 71.875 (79.092)
Epoch: [8][600/1562.5]	Time 0.268 (0.273)	Data 0.027 (0.032)	Loss 1.9474 (1.9200)	Prec@1 45.312 (47.600)	Prec@5 71.875 (78.819)
Epoch: [8][700/1562.5]	Time 0.267 (0.273)	Data 0.026 (0.032)	Loss 1.9267 (1.9301)	Prec@1 53.125 (47.417)	Prec@5 78.125 (78.669)
Epoch: [8][800/1562.5]	Time 0.268 (0.272)	Data 0.028 (0.031)	Loss 1.9168 (1.9340)	Prec@1 42.188 (47.339)	Prec@5 85.938 (78.531)
Epoch: [8][900/1562.5]	Time 0.270 (0.271)	Data 0.026 (0.030)	Loss 1.8012 (1.9362)	Prec@1 53.125 (47.281)	Prec@5 79.688 (78.527)
Epoch: [8][1000/1562.5]	Time 0.267 (0.271)	Data 0.027 (0.030)	Loss 2.1390 (1.9431)	Prec@1 45.312 (47.106)	Prec@5 73.438 (78.387)
Epoch: [8][1100/1562.5]	Time 0.267 (0.271)	Data 0.026 (0.030)	Loss 2.1406 (1.9498)	Prec@1 43.750 (46.92

Epoch: [11][1500/1562.5]	Time 0.262 (0.265)	Data 0.026 (0.028)	Loss 1.6818 (1.7998)	Prec@1 64.062 (50.423)	Prec@5 81.250 (81.190)
Test: [0/156.25]	Time 0.102 (0.102)	Loss 2.5817 (2.5817)	Prec@1 34.375 (34.375)	Prec@5 67.188 (67.188)
Test: [100/156.25]	Time 0.101 (0.102)	Loss 2.6387 (2.4638)	Prec@1 31.250 (37.206)	Prec@5 68.750 (69.369)
 * Prec@1 37.370 Prec@5 68.900
Epoch: [12][0/1562.5]	Time 0.265 (0.265)	Data 0.028 (0.028)	Loss 1.8166 (1.8166)	Prec@1 51.562 (51.562)	Prec@5 87.500 (87.500)
Epoch: [12][100/1562.5]	Time 0.266 (0.298)	Data 0.025 (0.060)	Loss 1.5726 (1.5958)	Prec@1 54.688 (56.219)	Prec@5 85.938 (84.793)
Epoch: [12][200/1562.5]	Time 0.262 (0.283)	Data 0.025 (0.044)	Loss 2.0571 (1.6266)	Prec@1 46.875 (55.177)	Prec@5 73.438 (84.289)
Epoch: [12][300/1562.5]	Time 0.263 (0.276)	Data 0.026 (0.038)	Loss 1.8253 (1.6465)	Prec@1 48.438 (54.444)	Prec@5 79.688 (83.970)
Epoch: [12][400/1562.5]	Time 0.263 (0.274)	Data 0.026 (0.035)	Loss 2.0200 (1.6661)	Prec@1 51.562 (53.912)	Prec@5 79.6

Epoch: [15][800/1562.5]	Time 0.267 (0.271)	Data 0.027 (0.030)	Loss 1.6523 (1.5793)	Prec@1 54.688 (55.739)	Prec@5 84.375 (85.040)
Epoch: [15][900/1562.5]	Time 0.268 (0.271)	Data 0.027 (0.030)	Loss 1.5595 (1.5901)	Prec@1 50.000 (55.471)	Prec@5 87.500 (84.904)
Epoch: [15][1000/1562.5]	Time 0.266 (0.270)	Data 0.026 (0.030)	Loss 1.9613 (1.5981)	Prec@1 51.562 (55.281)	Prec@5 78.125 (84.736)
Epoch: [15][1100/1562.5]	Time 0.267 (0.270)	Data 0.026 (0.029)	Loss 1.5416 (1.6066)	Prec@1 59.375 (55.027)	Prec@5 84.375 (84.567)
Epoch: [15][1200/1562.5]	Time 0.269 (0.270)	Data 0.028 (0.029)	Loss 1.8045 (1.6112)	Prec@1 45.312 (54.894)	Prec@5 82.812 (84.491)
Epoch: [15][1300/1562.5]	Time 0.266 (0.269)	Data 0.026 (0.029)	Loss 1.8275 (1.6162)	Prec@1 53.125 (54.774)	Prec@5 82.812 (84.394)
Epoch: [15][1400/1562.5]	Time 0.267 (0.269)	Data 0.026 (0.029)	Loss 1.8932 (1.6241)	Prec@1 43.750 (54.566)	Prec@5 79.688 (84.265)
Epoch: [15][1500/1562.5]	Time 0.266 (0.269)	Data 0.026 (0.028)	Loss 1.7457 (1.6297)	Prec@1 5

...

Epoch: [58][200/1562.5]	Time 0.263 (0.279)	Data 0.026 (0.042)	Loss 0.3257 (0.3223)	Prec@1 95.312 (92.436)	Prec@5 100.000 (99.176)
Epoch: [58][300/1562.5]	Time 0.262 (0.274)	Data 0.026 (0.037)	Loss 0.2731 (0.3251)	Prec@1 95.312 (92.385)	Prec@5 100.000 (99.149)
Epoch: [58][400/1562.5]	Time 0.263 (0.271)	Data 0.026 (0.034)	Loss 0.5177 (0.3238)	Prec@1 84.375 (92.429)	Prec@5 100.000 (99.190)
Epoch: [58][500/1562.5]	Time 0.263 (0.270)	Data 0.026 (0.033)	Loss 0.4197 (0.3245)	Prec@1 90.625 (92.375)	Prec@5 98.438 (99.223)
Epoch: [58][600/1562.5]	Time 0.263 (0.269)	Data 0.026 (0.032)	Loss 0.1750 (0.3230)	Prec@1 98.438 (92.408)	Prec@5 100.000 (99.249)
Epoch: [58][700/1562.5]	Time 0.263 (0.268)	Data 0.026 (0.031)	Loss 0.2286 (0.3221)	Prec@1 96.875 (92.417)	Prec@5 100.000 (99.242)
Epoch: [58][800/1562.5]	Time 0.263 (0.267)	Data 0.027 (0.030)	Loss 0.3089 (0.3224)	Prec@1 92.188 (92.420)	Prec@5 98.438 (99.243)
Epoch: [58][900/1562.5]	Time 0.263 (0.267)	Data 0.027 (0.030)	Loss 0.3703 (0.3229)	Prec@1 90

Epoch: [61][1200/1562.5]	Time 0.263 (0.266)	Data 0.026 (0.029)	Loss 0.2621 (0.3236)	Prec@1 96.875 (92.397)	Prec@5 100.000 (99.196)
Epoch: [61][1300/1562.5]	Time 0.263 (0.266)	Data 0.026 (0.028)	Loss 0.3107 (0.3241)	Prec@1 92.188 (92.375)	Prec@5 100.000 (99.199)
Epoch: [61][1400/1562.5]	Time 0.262 (0.265)	Data 0.025 (0.028)	Loss 0.2662 (0.3238)	Prec@1 93.750 (92.411)	Prec@5 100.000 (99.206)
Epoch: [61][1500/1562.5]	Time 0.263 (0.265)	Data 0.026 (0.028)	Loss 0.2657 (0.3243)	Prec@1 90.625 (92.393)	Prec@5 100.000 (99.199)
Test: [0/156.25]	Time 0.103 (0.103)	Loss 1.8790 (1.8790)	Prec@1 56.250 (56.250)	Prec@5 82.812 (82.812)
Test: [100/156.25]	Time 0.102 (0.102)	Loss 1.4540 (2.0431)	Prec@1 62.500 (50.975)	Prec@5 82.812 (79.254)
 * Prec@1 50.781 Prec@5 79.077
Epoch: [62][0/1562.5]	Time 0.265 (0.265)	Data 0.028 (0.028)	Loss 0.2533 (0.2533)	Prec@1 92.188 (92.188)	Prec@5 100.000 (100.000)
Epoch: [62][100/1562.5]	Time 0.262 (0.295)	Data 0.026 (0.058)	Loss 0.3269 (0.3335)	Prec@1 92.188 (92.265)	Pr

Epoch: [65][400/1562.5]	Time 0.268 (0.282)	Data 0.027 (0.039)	Loss 0.3343 (0.3279)	Prec@1 92.188 (92.293)	Prec@5 98.438 (99.135)
Epoch: [65][500/1562.5]	Time 0.267 (0.281)	Data 0.026 (0.037)	Loss 0.3037 (0.3247)	Prec@1 95.312 (92.421)	Prec@5 100.000 (99.167)
Epoch: [65][600/1562.5]	Time 0.269 (0.280)	Data 0.027 (0.036)	Loss 0.2207 (0.3233)	Prec@1 95.312 (92.468)	Prec@5 100.000 (99.189)
Epoch: [65][700/1562.5]	Time 0.281 (0.279)	Data 0.038 (0.035)	Loss 0.3579 (0.3228)	Prec@1 89.062 (92.437)	Prec@5 100.000 (99.193)
Epoch: [65][800/1562.5]	Time 0.268 (0.278)	Data 0.026 (0.035)	Loss 0.2470 (0.3219)	Prec@1 93.750 (92.447)	Prec@5 100.000 (99.196)
Epoch: [65][900/1562.5]	Time 0.280 (0.278)	Data 0.033 (0.034)	Loss 0.1905 (0.3218)	Prec@1 93.750 (92.486)	Prec@5 100.000 (99.206)
Epoch: [65][1000/1562.5]	Time 0.271 (0.277)	Data 0.027 (0.034)	Loss 0.6591 (0.3229)	Prec@1 84.375 (92.454)	Prec@5 96.875 (99.196)
Epoch: [65][1100/1562.5]	Time 0.274 (0.277)	Data 0.032 (0.034)	Loss 0.1777 (0.3236)	Prec@1 

Epoch: [68][1400/1562.5]	Time 0.262 (0.269)	Data 0.026 (0.031)	Loss 0.3594 (0.3242)	Prec@1 89.062 (92.353)	Prec@5 100.000 (99.196)
Epoch: [68][1500/1562.5]	Time 0.262 (0.269)	Data 0.025 (0.030)	Loss 0.2434 (0.3244)	Prec@1 98.438 (92.343)	Prec@5 100.000 (99.197)
Test: [0/156.25]	Time 0.103 (0.103)	Loss 1.6535 (1.6535)	Prec@1 56.250 (56.250)	Prec@5 82.812 (82.812)
Test: [100/156.25]	Time 0.102 (0.104)	Loss 2.5947 (2.0526)	Prec@1 50.000 (51.392)	Prec@5 71.875 (79.486)
 * Prec@1 50.982 Prec@5 79.377
Epoch: [69][0/1562.5]	Time 0.265 (0.265)	Data 0.028 (0.028)	Loss 0.4302 (0.4302)	Prec@1 89.062 (89.062)	Prec@5 98.438 (98.438)
Epoch: [69][100/1562.5]	Time 0.261 (0.294)	Data 0.025 (0.057)	Loss 0.3988 (0.3236)	Prec@1 90.625 (92.466)	Prec@5 100.000 (99.226)
Epoch: [69][200/1562.5]	Time 0.262 (0.278)	Data 0.025 (0.041)	Loss 0.3129 (0.3294)	Prec@1 92.188 (92.040)	Prec@5 100.000 (99.176)
Epoch: [69][300/1562.5]	Time 0.261 (0.272)	Data 0.025 (0.036)	Loss 0.2976 (0.3280)	Prec@1 90.625 (92.099)	Prec@5

KeyboardInterrupt: 

In [19]:
dummy_text_file = open("epoch_outputXX.txt", "w")
prec1 = validate(val_loader, model, criterion, dummy_text_file)

Test: [0/156.25]	Time 0.119 (0.119)	Loss 6.9066 (6.9066)	Prec@1 1.562 (1.562)	Prec@5 3.125 (3.125)
Test: [100/156.25]	Time 0.102 (0.102)	Loss 6.9088 (6.9075)	Prec@1 0.000 (1.006)	Prec@5 0.000 (1.083)
 * Prec@1 0.891 Prec@5 0.992


In [63]:
torch.save(model,'densenet_ep23_asymptotic.pt')

In [65]:
best_prec5

80.66907051282051

In [16]:
dense_model = torch.load('models/best_dense.pt')
dense_model2 = torch.load('models/best_dense201.pt')
resnet_model = torch.load('resnet34_ep200_asymptotic.pt')

In [17]:
def validate_ensamble(val_loader, model1, model2, model3, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(int(val_loader.size()/batch_size)):
        input, target = val_loader.next_batch(batch_size)
        target = target.long()
        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        target_var = target_var.long()

        # compute output
        output1 = model1(input_var)
        output2 = model2(input_var)
        output3 = model3(input_var)
        output = output3+output1+output2
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
                
        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))


    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top5.avg

In [18]:
validate_ensamble(val_loader, dense_model, dense_model, dense_model2, criterion)

Test: [0/156.25]	Time 0.872 (0.872)	Loss 2.8430 (2.8430)	Prec@1 57.812 (57.812)	Prec@5 85.938 (85.938)
Test: [100/156.25]	Time 0.191 (0.194)	Loss 3.3035 (3.6776)	Prec@1 59.375 (54.394)	Prec@5 85.938 (82.147)
 * Prec@1 54.137 Prec@5 82.302


82.3016826923077